In [1]:
# Import necessary libraries
import cv2  # OpenCV for computer vision tasks
import tkinter as tk  # Tkinter for GUI
from tkinter import messagebox, Menu  # Messagebox for error handling and Menu for adding menu bar
from threading import Thread  # Threading for running tasks in parallel
from PIL import Image, ImageTk  # PIL for image processing

In [2]:
def faceBox(faceNet, frame):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    
    # Create a blob from the frame to pass it through the network
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    
    # Perform forward pass to get the face detections
    detections = faceNet.forward()
    bboxs = []
    
    # Iterate over all detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        # Filter out weak detections by ensuring the confidence is greater than a threshold
        if confidence > 0.7:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            
            # Append the bounding box coordinates to the list
            bboxs.append([x1, y1, x2, y2])
            
            # Draw the bounding box on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return frame, bboxs


In [3]:
# Load model configuration and weights files for face, age, and gender detection
faceProto = "opencv_face_detector.pbtxt"  # Path to the face detector configuration file
faceModel = "opencv_face_detector_uint8.pb"  # Path to the face detector weights file
ageProto = "age_deploy.prototxt"  # Path to the age detector configuration file
ageModel = "age_net.caffemodel"  # Path to the age detector weights file
genderProto = "gender_deploy.prototxt"  # Path to the gender detector configuration file
genderModel = "gender_net.caffemodel"  # Path to the gender detector weights file

In [ ]:
try:
    faceNet = cv2.dnn.readNet(faceModel, faceProto)
    ageNet = cv2.dnn.readNet(ageModel, ageProto)
    genderNet = cv2.dnn.readNet(genderModel, genderProto)
    print("Models loaded successfully.")
except Exception as e:
    print(f"Error loading models: {e}")

In [5]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '3-5', '6-8', '9-12', '13-17', '18-22', '23-27', '28-32', '33-37', '38-42', '43-47', '48-52', '53-57', '58-62']
genderList = ['Male', 'Female']

In [ ]:
def start_camera():
    video = cv2.VideoCapture(0)
    video.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Set width to 1280 for 720p
    video.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Set height to 720 for 720p
    if not video.isOpened():
        messagebox.showerror("Error", "Failed to open camera!")
        return

    while True:
        ret, frame = video.read()
        if not ret:
            messagebox.showerror("Error", "Failed to read frame from camera!")
            break

        frame = cv2.flip(frame, 1)  # Fix the camera inverse mirroring

        # Detect faces
        frame, bboxs = faceBox(faceNet, frame)

        for bbox in bboxs:
            face = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
            
            # Predict gender
            genderNet.setInput(blob)
            genderPred = genderNet.forward()
            gender = genderList[genderPred[0].argmax()]

            # Predict age
            ageNet.setInput(blob)
            agePred = ageNet.forward()
            age = ageList[agePred[0].argmax()]

            label = "{},{}".format(gender, age)
            cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Convert the frame to RGB format
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)  # Convert OpenCV array to PIL Image

        # Maintain aspect ratio
        frame_width, frame_height = frame.size
        aspect_ratio = frame_width / frame_height
        new_width = 720
        new_height = int(new_width / aspect_ratio)
        frame = frame.resize((new_width, new_height))

        # Convert PIL Image to ImageTk format
        img = ImageTk.PhotoImage(image=frame)

        # Schedule the image update in the main thread using root.after
        root.after(0, update_image, img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

In [ ]:
def update_image(img):
    panel.config(image=img)
    panel.image = img  # Ensure the image remains referenced

# Function to handle the button press
def open_camera():
    # Run the camera in a separate thread
    global camera_thread
    camera_thread = Thread(target=start_camera)
    camera_thread.daemon = True  # Ensure the thread exits when the main program does
    camera_thread.start()

# Function to close the application
def exit_app():
    root.quit()  # Close the Tkinter application
    root.destroy()  # Destroy the Tkinter window

# Create the Tkinter window
root = tk.Tk()
root.title("Face Age & Gender Detection")

# Create a menu bar
menu_bar = Menu(root)
root.config(menu=menu_bar)

# Add a menu
file_menu = Menu(menu_bar, tearoff=0)
menu_bar.add_cascade(label="File", menu=file_menu)
file_menu.add_command(label="Open Camera", command=open_camera)
file_menu.add_separator()
file_menu.add_command(label="Exit", command=exit_app)

# Create a button to open the camera
open_button = tk.Button(root, text="Open Camera", width=20, height=2, command=open_camera)
open_button.pack(pady=10)

# Create a button to exit the application
exit_button = tk.Button(root, text="Exit", width=20, height=2, command=exit_app)
exit_button.pack(pady=10)

# Panel to display the video
panel = tk.Label(root)
panel.pack()

root.mainloop()